In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.widgets import CheckButtons
import globin

PSE = globin.atoms.PSE()

import ipywidgets as widgets
from IPython.display import display

In [10]:
path = "/home/dusan/Documents/globin_inversion/gris_aug18_trelles/obs/gris_20180829_003_TrellesArjona.p2.fits"
spectrum = globin.Observation(path, spec_type="hinode")

idx, idy = 10, 20

_, RLK_lines = globin.atoms.read_RLK_lines("/home/dusan/Documents/globin_inversion/gris_aug18_trelles/lines_trelles21_LS_lande_lam_ABO")

atoms = {}
for line in RLK_lines:
    element = PSE.get_element_symbol(line.ion)
    element += "I" if line.state==0 else "II"
    if element in atoms:
        atoms[element].append(line)
    else:
        atoms[element] = [line]

In [8]:
def callback(**flags):
    fig, ax = plt.subplots(nrows=1, ncols=1)

    ax.plot(spectrum.wavelength, spectrum.I[idx,idy])

    lines = {}
    for element in atoms:
        if not flags[element]:
            continue

        for line in atoms[element]:
            ax.axvline(x=line.lam0, c="grey", lw=0.75)

    ax.set_xlabel("wavelength [nm]")
    ax.set_ylabel("Normalized intensity")
    ax.set_xlim([spectrum.wavelength[0], spectrum.wavelength[-1]])
    
    fig.canvas.draw_idle()

In [11]:
args = {}
boxes = []
for element in atoms:
    checkbox = widgets.Checkbox(
        value=True, 
        description=element, 
        disabled=False,
        indent=True)
    boxes.append(checkbox)
    args[element] = checkbox
_boxes = widgets.HBox(boxes)

#----------

out = widgets.interactive_output(callback, args)

widgets.VBox([_boxes, out])